# Registration of confocal images of Drosophila abdomens 

This notebook runs the entire pipeline to register the confocal images of fly abdomens.
The analysis is divided in few steps. 
Each step uses functions saved in files in the src folder and its subfolders. 

The results of each step of the analysis are saved in a series of numbered subfolders in the data folder.

The raw data should be saved in the 01_raw subfolder as separate .tif files for each channel with a prefix that identifies the channel (like C1-filename.tif).  It should also contain an excel file with the columns "image file name", "construct" and "quality". These contain the name of each image stack, a label identifying which construct or genotype it is, and a label characterizing the quality of the raw data for filtering the results at a later point in the analysis.


In [1]:
# import packages 

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

## 1. Automated segmentation of the abdomens

In [8]:
# Importing the preprocessing function from the preprocess.py file:

from src.preprocess import preprocess_and_segment_images

raw_data_folder = "../../data_2/01_raw"
destination_folder = "../../data_2/02_preprocessed"

#downscaling of confocal stacks along z,x,y to make the resolution isotropic:
downscaling = (1, 2.5, 2.5) 

# bit depth of raw images (depending on detector settings may be 8, 12 or 16)
bit_depth = 12

# name of the excel file containing the list of images to be used in the analysis. 
db_name = 'DatasetInformation.xlsx'

preprocess_and_segment_images(raw_data_folder, destination_folder, downscaling, bit_depth, only_on_new_files = True, database_filename = db_name)

Preprocessing of raw images in progress:


100%|█████████████████████████████████████████████| 9/9 [03:43<00:00, 24.80s/it]


## 2. Manual registration of the segmented 3D images


In [9]:
# Importing the registration function from the registration.py file:

from src.registration import registration_of_abdomens_3D

"""
# USER INPUT IS REQUIRED, two windows with a 3D rendering of the detected fly abdomen
will pop-up for each image in the dataset. 
The first window shows the abdomen to register, the second one shows the reference abdomen.

For each couple of images:
1)  Please pick at least three correspondences using [shift + left click]
    Press [shift + right click] to undo point picking)
2)  After picking points, press 'Q' to close the window
3)  Repeat points 1 and 2 for the reference image in the second window.
4)  The next window will show the result of the registration, press 'Q' to close the window.

"""

read_folder = "../../data_2/02_preprocessed"
destination_folder = "../../data_2/03_registered"

reference_fly_filename = "../../data_2/References_and_masks/C1_Reference_iso.tif"
abdomen_mask_file = "../../data_2/References_and_masks/Reference_abdomen_mask_iso.tif"

df_name = "DatasetInformation.xlsx"

preprocessed_df = pd.read_excel(os.path.join(read_folder,df_name))
registration_of_abdomens_3D(preprocessed_df, read_folder, reference_fly_filename, abdomen_mask_file, destination_folder, only_on_new_files = True)

Registration of 3D stacks in progress:


100%|█████████████████████████████████████████████| 9/9 [04:04<00:00, 27.12s/it]


## 3. Automated 2D projection of registered 3D images 

In [2]:
from src.projection import projections_of_abdomens

read_folder = "../../data_2/03_registered"
destination_folder = "../../data_2/04_projected"
landmark_folder = "../../data_2/05_landmarks/data"
abdomen_mask_file = "../../data_2/References_and_masks/Reference_abdomen_mask_iso_thick.tif"
abdomen_shape_reference_file = "../../data_2/References_and_masks/Reference_abdomen_mask_iso.tif"

df_name = "DatasetInformation.xlsx"
df = pd.read_excel(os.path.join(read_folder,df_name))
projections_of_abdomens(df, read_folder, destination_folder, landmark_folder, abdomen_mask_file, abdomen_shape_reference_file, crop_x=300, crop_y = 320)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Projection of registered 3D stacks to 2D images in progress:


100%|█████████████████████████████████████████████| 9/9 [02:19<00:00, 15.53s/it]


##  4. Annotation and warping of the 2D projected images

After the projection from 3D to 2D, images are already roughly aligned. However, the alignment up to this point was only performed by applying a global rescaling and rigid rotations/translations of the original stacks.

To fine tune the registration we need to perform an elastic registration of the final images. We can do this using a separate software for 2D image annotation and registration.

Run the image annotation software, open the annotation project, add the new images found in the folder 05_landmarks/data to the project. Annotate the new images and run the registration, save the results in the folder 06_warped.

In [4]:
import image_registration

image_registration.gui.start(main_window_size = (1000, 1000), graph_canvas_width = 600)

## 5. Final touch: smoothing and masking


In [2]:
from src.masking_smoothing import preprocess_registered_images

destination_folder = "../../data_2/07_masked_and_smooth"
read_folder = "../../data_2/06_warped"
mask = '../../data_2/References_and_masks/mask.tif'
database_registered_images = "../../data_2/06_warped/dataframe_info.csv"
database_info = "../../data_2/04_projected/DatasetInformation.xlsx"

preprocess_registered_images(read_folder, destination_folder, database_registered_images, database_info, mask_filename = mask, smooth_x=1, smooth_y = 5, binning=1)

Preprocessing of registered raw images in progress:


100%|████████████████████████████████████████| 123/123 [00:01<00:00, 102.25it/s]
